In [1]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt
!pip install openpyxl


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
data = pd.read_excel(r'Raw Game Data/06-04-2023.xlsx', index_col = "Game Pitch") 
data = data.drop(columns = ["ClientDeviceID", "Electron App Version", "Device Manager Version", "FirmwareVersion", "Field ID", "Field Name", "RHostLibVersion", "STMVersion", "ServerDeviceID", "Server Video 1", "Client Video 1", "Server Video 2", "Client Video 2", "Pitch PlateSpin (RPM)", "Pitch TrueSpinRelease (RPM)", "Pitch SpinEfficiencySZ (%)", "Pitch HorizontalBreakSpin (Inches)", "Pitch VerticalBreakSpin (Inches)", "Pitch GyroDegree (Degrees)", "Pitch OriginSpinX (RPM)", "Pitch OriginSpinY (RPM)", "Pitch OriginSpinZ (RPM)", "Pitch LegacySpinX (RPM)", "Pitch LegacySpinY (RPM)", "Pitch LegacySpinZ (RPM)", "Pitch PositionX (Feet)", "Pitch PositionY (Feet)", "Pitch PositionZ (Feet)", "Pitch StartTime (Seconds)", "Pitch EndTime (Seconds)", "Pitch PlateSpin (RPM)", "Hit VelocityX (Ft/Sec)", "Hit VelocityY (Ft/Sec)", "Hit VelocityZ (Ft/Sec)", "Hit SpinX (RPM)", "Hit SpinY (RPM)", "Hit SpinZ (RPM)", "Hit BallType", "Hit SpinTiltMinute", "Hit EffectiveSpin (RPM)", "Hit SpinEfficiency (%)", "Hit HitClass", "Hit OriginSpinX (RPM)", "Hit OriginSpinY (RPM)", "Hit OriginSpinZ (RPM)", "Hit SpinTiltHour", "Hit PointofLaunchX (Feet)", "Hit PointofLaunchY (Feet)", "Hit PointofLaunchZ (Feet)", "Hit SpinAxisX (RPM)", "Hit SpinAxisY (RPM)", "Hit SpinAxisZ (RPM)", "Hit Incoming Speed (MPH)", "Hit TotalSpin (RPM)", "Hit StrikePositionX (Inches)", "Hit StrikePositionY (Inches)"])
data

FileNotFoundError: [Errno 2] No such file or directory: 'Raw Game Data/06-04-2023.xlsx'

In [ ]:
data['Pitch TotalSpeed (MPH)'].replace('-', np.nan, inplace=True)  
data['Pitch TotalSpeed (MPH)'].replace(0, np.nan, inplace=True) 
data.dropna(subset = ['Pitch TotalSpeed (MPH)'], inplace =True)  
data

In [ ]:
grouped = data.groupby('Pitcher ID')
grouped

In [ ]:
pitchers = data[['Pitcher First Name', 'Pitcher Last Name', 'Pitcher ID']] 
pitchers

In [ ]:
num_pitchers = len(data['Pitcher ID'].unique())
fig, axes = plt.subplots(nrows=num_pitchers, ncols=1, figsize=(8, 4 * num_pitchers))

for i, (pitcher_id, group) in enumerate(data.groupby('Pitcher ID')):
    pitcher_first_name = pitchers.loc[pitchers['Pitcher ID'] == pitcher_id, 'Pitcher First Name'].iloc[0]
    pitcher_last_name = pitchers.loc[pitchers['Pitcher ID'] == pitcher_id, 'Pitcher Last Name'].iloc[0]
    pitcher_name = f"{pitcher_first_name} {pitcher_last_name}"
    ax = axes[i]  # Get the appropriate subplot for the pitcher
    group.plot(x='Pitch Number', y='Pitch TotalSpeed (MPH)', ax=ax, label=f'Pitcher {pitcher_name}')
    
    # Add markers for hits and strikeouts
    hits = group[(group['Hit Outcomes'].notnull()) & (group['Hit Outcomes'] != 'BB') & (group['Hit Outcomes'] != 'E') & (group['Hit Outcomes'].isin(['1B', '2B', '3B', 'HR']))]
    strikeouts = group[group['Hit Outcomes'] == 'K']
    
    if not hits.empty:
        ax.scatter(hits['Pitch Number'], hits['Pitch TotalSpeed (MPH)'], color='red', marker='o', label='Hit')
    
    if not strikeouts.empty:
        ax.scatter(strikeouts['Pitch Number'], strikeouts['Pitch TotalSpeed (MPH)'], color='blue', marker='x', label='Strikeout')
    
    ax.set_xlabel('Pitch #')
    ax.set_ylabel('Pitch TotalSpeed (MPH)')
    ax.set_title(f'Velocity of Fastballs - {pitcher_name}')
    ax.legend()

plt.tight_layout() 

file_path = 'Image Files/06-04-2023/Velocity_of_Fastballs_06-04-2023.png'
plt.savefig(file_path)    
    
plt.show() 


In [ ]:
# Create subplots
fig, axes = plt.subplots(nrows=num_pitchers, figsize=(8, 4*num_pitchers))

# Iterate over each subplot and pitcher ID
for i, (pitcher_id, group) in enumerate(data.groupby('Pitcher ID')):
    # Get the pitcher's name from the pitchers DataFrame
    pitcher_first_name = pitchers.loc[pitchers['Pitcher ID'] == pitcher_id, 'Pitcher First Name'].iloc[0]
    pitcher_last_name = pitchers.loc[pitchers['Pitcher ID'] == pitcher_id, 'Pitcher Last Name'].iloc[0]
    pitcher_name = f"{pitcher_first_name} {pitcher_last_name}"

    # Get the data for the current pitcher
    pitcher_data = data[data['Pitcher ID'] == pitcher_id]

    # Group the data by pitch type
    grouped = pitcher_data.groupby('Pitch Type')

    # Get the count of each pitch type
    pitch_type_counts = grouped.size()

    # Calculate the percentage of each pitch type
    pitch_type_percentages = pitch_type_counts / pitch_type_counts.sum()

    # Plot the pie chart
    ax = axes[i]
    ax.pie(pitch_type_percentages, labels=pitch_type_percentages.index, autopct='%1.1f%%')
    ax.set_title(f'{pitcher_name} - Pitch Type Distribution')

file_path2 = 'Image Files/06-13-2023/Pitch_Type_Distribution_06-13-2023.png'
plt.savefig(file_path2)    

# Adjust the layout to avoid overlapping subplots
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
# Create subplots
fig, axes = plt.subplots(nrows=num_pitchers, figsize=(10, 6*num_pitchers))

# Iterate over each subplot and pitcher ID
for i, (pitcher_id, group) in enumerate(data.groupby('Pitcher ID')):
    # Get the pitcher's name from the pitchers DataFrame 
    pitcher_first_name = pitchers.loc[pitchers['Pitcher ID'] == pitcher_id, 'Pitcher First Name'].iloc[0]
    pitcher_last_name = pitchers.loc[pitchers['Pitcher ID'] == pitcher_id, 'Pitcher Last Name'].iloc[0]
    pitcher_name = f"{pitcher_first_name} {pitcher_last_name}"
    
    # Get the data for the current pitcher
    pitcher_data = data[data['Pitcher ID'] == pitcher_id]
    
    # Group the data by pitch type
    grouped = pitcher_data.groupby('Pitch Type')
    
    # Get the corresponding subplot for the pitcher
    ax = axes[i]
    
    # Iterate over each pitch type
    for pitch_type, group in grouped:
        # Filter out hit outcomes equal to 'BB'
        group_filtered = group[group['Hit Outcomes'] != 'BB']
        
        # Plot the pitch velocity
        ax.plot(group_filtered['Pitch Number'], group_filtered['Pitch TotalSpeed (MPH)'], label=pitch_type)
        
        # Filter hits for '1B', '2B', '3B'
        hits = group[(group['Hit Outcomes'].notnull()) & (group['Hit Outcomes'] != 'BB') & (group['Hit Outcomes'] != 'E') & (group['Hit Outcomes'].isin(['1B', '2B', '3B', 'HR']))]
        
        # Filter strikeouts
        strikeouts = group_filtered[group_filtered['Hit Outcomes'] == 'K']
        
        # Add markers for hits
        if not hits.empty:
            ax.scatter(hits['Pitch Number'], hits['Pitch TotalSpeed (MPH)'], color='red', marker='o', label='Hit')
        
        # Add markers for strikeouts
        if not strikeouts.empty:
            ax.scatter(strikeouts['Pitch Number'], strikeouts['Pitch TotalSpeed (MPH)'], color='blue', marker='X', label='Strikeout')
        
        # Add labels for each pitch number
        for index, pitch in group_filtered.iterrows():
            ax.annotate(pitch['Pitch Number'], (pitch['Pitch Number'], pitch['Pitch TotalSpeed (MPH)']),
                        textcoords="offset points", xytext=(-10, 10), ha='center')
    
    # Set the x and y-axis labels
    ax.set_xlabel('Pitch Number')
    ax.set_ylabel('Pitch TotalSpeed (MPH)')
    
    # Set the plot title as the pitcher's name
    ax.set_title(f'{pitcher_name}')
    
    # Add a legend
    ax.legend()

# Adjust the layout to avoid overlapping subplots
plt.tight_layout()

# Save the plot as an image file
file_path = 'Image Files/06-04-2023/Velocity_of_Pitch_Types_06-04-2023.png'
plt.savefig(file_path)

# Display the plot
plt.show()


In [ ]:
import img2pdf

output_pdf_path = b'Image Files/06-04-2023/Game_Data_06-04-2023.pdf'

# Convert images to PDF
with open(output_pdf_path, "wb") as f:
    f.write(img2pdf.convert(
        ['Image Files/06-04-2023/Pitch_Type_Distribution_06-04-2023.png', 
        'Image Files/06-04-2023/Velocity_of_Fastballs_06-04-2023.png', 
        'Image Files/06-04-2023/Velocity_of_Pitch_Types_06-04-2023.png']))
